# Quantum State Preparation and Entanglement Analysis

## Introduction

This implementation provides a comprehensive framework for quantum state preparation and entanglement analysis using fundamental linear algebra operations. The code demonstrates how to construct quantum state vectors from complex amplitudes while adhering to the mathematical principles of quantum mechanics, without relying on high-level quantum computing libraries.

## State Preparation Implementation

The core functionality revolves around preparing normalized quantum state vectors from given complex amplitudes. For a two-qubit system, we take four complex numbers representing the amplitudes for the |00⟩, |01⟩, |10⟩, and |11⟩ basis states. The implementation automatically checks and enforces normalization by calculating the Euclidean norm of the input amplitudes and scaling them appropriately if needed. This ensures that the total probability sums to one, satisfying the fundamental requirement of quantum state vectors.

The same approach extends to three-qubit systems, where eight complex amplitudes define the state in the computational basis from |000⟩ to |111⟩. The code includes robust error handling to validate input dimensions and uses numerical tolerances to account for floating-point precision issues that commonly arise in numerical computations.

## Schmidt Decomposition and Entanglement Analysis

A significant enhancement in this implementation is the incorporation of Schmidt decomposition for entanglement analysis. The Schmidt decomposition provides a powerful mathematical tool to analyze bipartite quantum systems by expressing the state in terms of orthogonal basis states for each subsystem. This is achieved through singular value decomposition (SVD) of the state vector reshaped into an appropriate matrix form.

The implementation computes Schmidt coefficients, which are the singular values obtained from the SVD, and determines the Schmidt rank by counting non-zero coefficients. The Schmidt rank serves as a crucial indicator of entanglement: a rank of one signifies a product state that can be factored into separate states for each subsystem, while higher ranks indicate varying degrees of entanglement. For example, a Bell state exhibits Schmidt rank two with equal coefficients, representing maximal entanglement between two qubits.

The code includes functionality to automatically detect product states and analyze entanglement patterns in multi-qubit systems. For three-qubit states, different bipartitions can be examined, such as separating one qubit from the other two, revealing the complex entanglement structure that emerges in larger quantum systems.

## Tensor Product Operations

The implementation demonstrates how composite quantum systems are constructed through tensor products of smaller subsystems. The tensor product operation, implemented using the Kronecker product, shows how individual qubit states combine to form multi-qubit states. This foundational operation illustrates the principle that the state space of a composite system is the tensor product of the state spaces of the component systems.

Through examples like |0⟩ ⊗ |1⟩ = |01⟩ and |+⟩ ⊗ |+⟩, the code shows how familiar single-qubit states combine to create more complex multi-qubit states. This operation is essential for understanding how quantum systems scale and how entanglement can be created through various quantum operations.

## Testing and Verification

A comprehensive testing suite validates all aspects of the implementation. Unit tests verify normalization correctness, output dimensions, and mathematical properties of prepared states. The tests cover various scenarios including already normalized states, unnormalized inputs, complex-valued states, and special quantum states like Bell and GHZ states.

The testing framework also validates the entanglement analysis by checking that product states correctly yield Schmidt rank one and entangled states show higher ranks. Tensor product operations are verified against known results to ensure mathematical correctness. These tests provide confidence in the implementation's accuracy and robustness.

## Mathematical Foundation

The implementation bridges abstract quantum mechanical concepts with concrete linear algebra operations. The state vector representation follows the standard quantum notation |ψ⟩ = Σ aᵢ|i⟩ where the coefficients aᵢ are complex probability amplitudes. The normalization condition |a₀|² + |a₁|² + ... = 1 ensures proper probability interpretation.

The Schmidt decomposition implementation showcases the deep connection between quantum entanglement and linear algebra through the singular value decomposition. By reshaping the state vector into a matrix and performing SVD, we obtain the Schmidt coefficients that quantify entanglement between subsystems. This mathematical approach provides an operational method to analyze quantum correlations without requiring quantum gates or circuits.

## Practical Applications

This implementation serves as an educational tool for understanding quantum state representation and entanglement properties. It demonstrates how fundamental quantum concepts can be implemented using standard numerical computing techniques, making the abstract mathematics of quantum mechanics more accessible and concrete.

The code provides a foundation for more advanced quantum information processing tasks, such as entanglement quantification, state tomography, and quantum circuit design. By building these capabilities from first principles, the implementation offers insights that might be obscured when using high-level quantum computing libraries.

## Conclusion

This quantum state preparation framework successfully implements core quantum mechanical concepts using fundamental linear algebra operations. The addition of Schmidt decomposition and tensor product operations provides a comprehensive toolkit for quantum state analysis, enabling detailed study of entanglement properties in multi-qubit systems. The implementation maintains mathematical rigor while being practically useful for both educational purposes and foundational quantum information processing tasks.

In [2]:
import numpy as np
import math

def prepare_two_qubit_state(amplitudes):
    """
    Prepare a two-qubit quantum state from given complex amplitudes.
    
    Args:
        amplitudes: List of 4 complex numbers [a0, a1, a2, a3] representing
                   the amplitudes for |00⟩, |01⟩, |10⟩, |11⟩ basis states.
    
    Returns:
        numpy.array: Normalized 2-qubit state vector as a complex numpy array
    """
    if len(amplitudes) != 4:
        raise ValueError("Two-qubit state requires exactly 4 amplitudes")
    
    # Convert to numpy array for easier computation
    state = np.array(amplitudes, dtype=complex)
    
    # Calculate norm
    norm = math.sqrt(sum(abs(amp)**2 for amp in state))
    
    # Normalize if necessary
    if abs(norm - 1.0) > 1e-10:  # Small tolerance for floating point errors
        state = state / norm
    
    return state

def prepare_three_qubit_state(amplitudes):
    """
    Prepare a three-qubit quantum state from given complex amplitudes.
    
    Args:
        amplitudes: List of 8 complex numbers [a0, a1, a2, a3, a4, a5, a6, a7]
                   representing amplitudes for |000⟩, |001⟩, |010⟩, |011⟩,
                   |100⟩, |101⟩, |110⟩, |111⟩ basis states.
    
    Returns:
        numpy.array: Normalized 3-qubit state vector as a complex numpy array
    """
    if len(amplitudes) != 8:
        raise ValueError("Three-qubit state requires exactly 8 amplitudes")
    
    # Convert to numpy array for easier computation
    state = np.array(amplitudes, dtype=complex)
    
    # Calculate norm
    norm = math.sqrt(sum(abs(amp)**2 for amp in state))
    
    # Normalize if necessary
    if abs(norm - 1.0) > 1e-10:  # Small tolerance for floating point errors
        state = state / norm
    
    return state

def compute_schmidt_coefficients(state_vector, partition=None):
    """
    Compute Schmidt coefficients for a bipartite quantum system.
    
    Args:
        state_vector: The quantum state vector
        partition: Tuple (dim_A, dim_B) specifying the partition.
                  For 2-qubit: typically (2, 2)
                  For 3-qubit: can be (2, 4) or (4, 2) etc.
    
    Returns:
        tuple: (schmidt_coefficients, schmidt_rank)
    """
    if partition is None:
        # Default: equal partition for even qubit systems
        dim = len(state_vector)
        if dim == 4:  # 2-qubit
            partition = (2, 2)
        elif dim == 8:  # 3-qubit
            partition = (2, 4)  # 1 qubit vs 2 qubits
        else:
            raise ValueError("Please specify partition for non-standard dimensions")
    
    dim_A, dim_B = partition
    
    if dim_A * dim_B != len(state_vector):
        raise ValueError(f"Partition dimensions {dim_A}×{dim_B} = {dim_A*dim_B} "
                        f"don't match state dimension {len(state_vector)}")
    
    # Reshape state vector into matrix for Schmidt decomposition
    state_matrix = state_vector.reshape(dim_A, dim_B)
    
    # Perform SVD to get Schmidt coefficients
    U, S, Vh = np.linalg.svd(state_matrix)
    
    # Schmidt coefficients are the singular values
    schmidt_coefficients = S
    schmidt_rank = np.sum(S > 1e-10)  # Count non-zero coefficients
    
    return schmidt_coefficients, schmidt_rank

def is_product_state(state_vector, partition=None, tolerance=1e-10):
    """
    Check if a state is a product state using Schmidt decomposition.
    
    A state is a product state if it has only one non-zero Schmidt coefficient.
    """
    schmidt_coeffs, schmidt_rank = compute_schmidt_coefficients(state_vector, partition)
    return schmidt_rank == 1

def tensor_product(state1, state2):
    """
    Compute the tensor product of two quantum state vectors.
    
    Args:
        state1, state2: State vectors as numpy arrays
    
    Returns:
        numpy.array: Tensor product state1 ⊗ state2
    """
    return np.kron(state1, state2)

# Unit tests
def test_two_qubit_state():
    """Test cases for two-qubit state preparation"""
    
    # Test 1: Already normalized state
    amplitudes1 = [0.5, 0.5, 0.5, 0.5]  # This sums to 1 when squared
    state1 = prepare_two_qubit_state(amplitudes1)
    assert len(state1) == 4, "State vector should have dimension 4"
    assert abs(np.linalg.norm(state1) - 1.0) < 1e-10, "State should be normalized"
    
    # Test 2: Unnormalized state
    amplitudes2 = [1.0, 1.0, 1.0, 1.0]  # Norm = 2.0, should be normalized to 0.5 each
    state2 = prepare_two_qubit_state(amplitudes2)
    assert len(state2) == 4, "State vector should have dimension 4"
    assert abs(np.linalg.norm(state2) - 1.0) < 1e-10, "State should be normalized"
    expected2 = [0.5, 0.5, 0.5, 0.5]
    for i in range(4):
        assert abs(state2[i] - expected2[i]) < 1e-10, f"Amplitude {i} incorrect"
    
    # Test 3: Complex amplitudes
    amplitudes3 = [1/math.sqrt(2), 0, 0, 1/math.sqrt(2)]  # Bell state
    state3 = prepare_two_qubit_state(amplitudes3)
    assert len(state3) == 4, "State vector should have dimension 4"
    assert abs(np.linalg.norm(state3) - 1.0) < 1e-10, "State should be normalized"
    
    print("All two-qubit tests passed!")

def test_three_qubit_state():
    """Test cases for three-qubit state preparation"""
    
    # Test 1: GHZ state
    amplitudes1 = [1/math.sqrt(2), 0, 0, 0, 0, 0, 0, 1/math.sqrt(2)]
    state1 = prepare_three_qubit_state(amplitudes1)
    assert len(state1) == 8, "State vector should have dimension 8"
    assert abs(np.linalg.norm(state1) - 1.0) < 1e-10, "State should be normalized"
    
    # Test 2: Unnormalized state
    amplitudes2 = [1.0] * 8  # Norm = sqrt(8) = 2√2
    state2 = prepare_three_qubit_state(amplitudes2)
    assert len(state2) == 8, "State vector should have dimension 8"
    assert abs(np.linalg.norm(state2) - 1.0) < 1e-10, "State should be normalized"
    
    print("All three-qubit tests passed!")

def test_schmidt_decomposition():
    """Test Schmidt decomposition for entanglement analysis"""
    
    # Test 1: Product state (should have Schmidt rank 1)
    # |00⟩ state
    product_state = prepare_two_qubit_state([1, 0, 0, 0])
    schmidt_coeffs, rank = compute_schmidt_coefficients(product_state)
    assert rank == 1, "Product state should have Schmidt rank 1"
    assert is_product_state(product_state), "|00⟩ should be a product state"
    
    # Test 2: Bell state (maximally entangled, should have Schmidt rank 2)
    bell_state = prepare_two_qubit_state([1/math.sqrt(2), 0, 0, 1/math.sqrt(2)])
    schmidt_coeffs, rank = compute_schmidt_coefficients(bell_state)
    assert rank == 2, "Bell state should have Schmidt rank 2"
    assert not is_product_state(bell_state), "Bell state should not be a product state"
    # Check that Schmidt coefficients are equal (maximal entanglement)
    assert abs(schmidt_coeffs[0] - 1/math.sqrt(2)) < 1e-10
    assert abs(schmidt_coeffs[1] - 1/math.sqrt(2)) < 1e-10
    
    # Test 3: Three-qubit product state
    three_qubit_product = prepare_three_qubit_state([1, 0, 0, 0, 0, 0, 0, 0])  # |000⟩
    schmidt_coeffs, rank = compute_schmidt_coefficients(three_qubit_product, (2, 4))
    assert rank == 1, "Three-qubit product state should have Schmidt rank 1"
    
    print("All Schmidt decomposition tests passed!")

def test_tensor_product():
    """Test tensor product operations"""
    
    # Single qubit states
    zero = np.array([1, 0], dtype=complex)
    one = np.array([0, 1], dtype=complex)
    plus = np.array([1/math.sqrt(2), 1/math.sqrt(2)], dtype=complex)
    
    # Test |0⟩ ⊗ |0⟩ = |00⟩
    state_00 = tensor_product(zero, zero)
    expected_00 = prepare_two_qubit_state([1, 0, 0, 0])
    assert np.allclose(state_00, expected_00), "Tensor product |0⟩⊗|0⟩ should equal |00⟩"
    
    # Test |0⟩ ⊗ |1⟩ = |01⟩
    state_01 = tensor_product(zero, one)
    expected_01 = prepare_two_qubit_state([0, 1, 0, 0])
    assert np.allclose(state_01, expected_01), "Tensor product |0⟩⊗|1⟩ should equal |01⟩"
    
    # Test |+⟩ ⊗ |+⟩
    state_plus_plus = tensor_product(plus, plus)
    expected_plus_plus = prepare_two_qubit_state([0.5, 0.5, 0.5, 0.5])
    assert np.allclose(state_plus_plus, expected_plus_plus), "Tensor product |+⟩⊗|+⟩ incorrect"
    
    print("All tensor product tests passed!")

if __name__ == "__main__":
    # Run tests
    test_two_qubit_state()
    test_three_qubit_state()
    test_schmidt_decomposition()
    test_tensor_product()
    
    # Example usage
    print("\nExample two-qubit state (Bell state):")
    bell_state = prepare_two_qubit_state([1/math.sqrt(2), 0, 0, 1/math.sqrt(2)])
    print(f"State vector: {bell_state}")
    print(f"Norm: {np.linalg.norm(bell_state)}")
    
    # Schmidt analysis
    schmidt_coeffs, rank = compute_schmidt_coefficients(bell_state)
    print(f"Schmidt coefficients: {schmidt_coeffs}")
    print(f"Schmidt rank: {rank}")
    print(f"Is product state: {is_product_state(bell_state)}")
    
    print("\nExample three-qubit state (GHZ state):")
    ghz_state = prepare_three_qubit_state([1/math.sqrt(2), 0, 0, 0, 0, 0, 0, 1/math.sqrt(2)])
    print(f"State vector: {ghz_state}")
    print(f"Norm: {np.linalg.norm(ghz_state)}")
    
    # Schmidt analysis for 3-qubit state (1 qubit vs 2 qubits)
    schmidt_coeffs_3q, rank_3q = compute_schmidt_coefficients(ghz_state, (2, 4))
    print(f"Schmidt coefficients (partition 2×4): {schmidt_coeffs_3q}")
    print(f"Schmidt rank: {rank_3q}")
    
    print("\nTensor product example:")
    zero = np.array([1, 0])
    one = np.array([0, 1]) 
    print(f"|0⟩ ⊗ |1⟩ = {tensor_product(zero, one)}")

All two-qubit tests passed!
All three-qubit tests passed!
All Schmidt decomposition tests passed!
All tensor product tests passed!

Example two-qubit state (Bell state):
State vector: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Norm: 0.9999999999999999
Schmidt coefficients: [0.70710678 0.70710678]
Schmidt rank: 2
Is product state: False

Example three-qubit state (GHZ state):
State vector: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Norm: 0.9999999999999999
Schmidt coefficients (partition 2×4): [0.70710678 0.70710678]
Schmidt rank: 2

Tensor product example:
|0⟩ ⊗ |1⟩ = [0 1 0 0]
